In [ ]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In the previous notebooks, we show how to preprocess a dataframe with single-hot categorical columns, and train a Tensorflow model with processed parquet files, and then to send request to Triton Inference Server. In this notebook, we deal with multi-hot categorical columns. To learn more about on multi-hot example with movielens dataset, you can visit the example notebooks under the [getting-started-movielens](https://github.com/NVIDIA/NVTabular/tree/main/examples/getting-started-movielens) folder.

### Learning Objectives

In this notebook, we show how to
- join two dataframes with JoinExternal operator
- preprocess single-hot categorical input features with NVTabular
- preprocess multi-hot categorical input features with NVTabular
- use LambdaOp for custom row-wise dataframe manipulations with NVTabular
- train a TF MLP model and save it on disk
- save the ensemble model to be used at the inference stage

## Getting Started

In [ ]:
import os

import tensorflow as tf
from tensorflow.python.feature_column import feature_column_v2 as fc

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# TF will have claimed all free GPU memory
os.environ['TF_MEMORY_ALLOCATION'] = "0.6" # fraction of free memory
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater
from nvtabular.framework_utils.tensorflow import layers
from tensorflow.python.feature_column import feature_column_v2 as fc

In [ ]:
# External dependencies
import cudf                 # cuDF is an implementation of Pandas-like Dataframe on GPU
import time
import gc
import shutil

import nvtabular as nvt

from os import path
from sklearn.model_selection import train_test_split

We define our base directory, containing the data.

In [ ]:
# path to store raw and preprocesses data
BASE_DIR = os.environ.get('BASE_DIR', '/model/data/')

If the data is not available in the base directory, we will download and unzip the data.

In [ ]:
if not path.exists(BASE_DIR + 'ml-25m'):
    if not path.exists(BASE_DIR + 'ml-25m.zip'):
        os.system("wget http://files.grouplens.org/datasets/movielens/ml-25m.zip")
        os.system("mv ml-25m.zip " + BASE_DIR)
    os.system("unzip " + BASE_DIR + "ml-25m.zip -d " + BASE_DIR)

## Preparing the dataset with NVTabular

First, we take a look on the movie metadata. 

We load the movies.

In [ ]:
movies= cudf.read_csv(os.path.join(BASE_DIR, "ml-25m", "movies.csv"))
movies.head()

In [ ]:
movies['genres'] = movies['genres'].str.split('|')
movies = movies.drop('title', axis=1)
movies.head()

We load the movie ratings.

In [ ]:
ratings = cudf.read_csv(os.path.join(BASE_DIR, "ml-25m", "ratings.csv"))
ratings.head()

We drop the timestamp column and split the ratings into training and test dataset. We use a simple random split.

In [ ]:
ratings = ratings.drop('timestamp', axis=1)
train, valid = train_test_split(ratings, test_size=0.2, random_state=42)

In [ ]:
train.to_parquet(BASE_DIR + 'train.parquet')
valid.to_parquet(BASE_DIR + 'valid.parquet')

Embedding Layers of neural networks require, that categorical features are continuous, incremental Integers: 0, 1, 2, ... , |C|-1. We need to ensure that our categorical features fulfill the requirement. We should transform the single-hot categorical features userId and movieId, and multi-hot categorical feature genres. NVTabular provides the operator `Categorify`, which provides this functionality with a high-level API out of the box.

Currently, our dataset consists of two separate dataframes. First, we use the JoinExternal operator to left-join the metadata (genres) to our rating dataset.

In [ ]:
joined = (
    ['userId', 'movieId'] >> 
    nvt.ops.JoinExternal(movies, on=['movieId'])
)

Next, we will add Categorify for our categorical features (single hot: userId, movieId and multi-hot: genres).

In [ ]:
cat_features = joined >> nvt.ops.Categorify(cat_cache="device")

The ratings are on a scale between 1-5. We want to predict a binary target with 1 are all ratings `>=4` and 0 are all ratings `<=3`. We use the [LambdaOp](https://github.com/NVIDIA/NVTabular/blob/main/nvtabular/ops/lambdaop.py) for it.

In [ ]:
ratings = nvt.ColumnGroup(['rating']) >> (lambda col: (col>3).astype('int8'))

We can visualize our calculation graph.

In [ ]:
output = cat_features+ratings
(output).graph

We initialize our NVTabular `workflow`.

In [ ]:
workflow = nvt.Workflow(output)

We save our train and valid datasets as parquet files on disk, and below we will read them in while initializing the Dataset objects.

In [ ]:
del train
del valid
gc.collect()

We initialize NVTabular Datasets, and use the `part_size` parameter, which defines the size read into GPU-memory at once, in `nvt.Dataset`.

In [ ]:
train_iter = nvt.Dataset(BASE_DIR + 'train.parquet', part_size='100MB')
valid_iter = nvt.Dataset(BASE_DIR + 'valid.parquet', part_size='100MB')

First, we collect the training dataset statistics.

In [ ]:
%%time

workflow.fit(train_iter)

We clear our output directories.

In [ ]:
if path.exists(os.path.join(BASE_DIR, "train")): 
    shutil.rmtree(os.path.join(BASE_DIR, "train"))
if path.exists(os.path.join(BASE_DIR, "valid")): 
    shutil.rmtree(os.path.join(BASE_DIR, "valid"))

We transform our workflow with `.transform`.

In [ ]:
%time

workflow.transform(train_iter).to_parquet(output_path=BASE_DIR + 'train/')
workflow.transform(valid_iter).to_parquet(output_path=BASE_DIR + 'valid/', shuffle=False)

We can take a look in the output dir.

In [ ]:
# !ls $BASE_DIR/train

## TensorFlow: Training Neural Network

### Reviewing data

We can take a look on the data.

In [ ]:
import glob

TRAIN_PATHS = sorted(glob.glob(BASE_DIR + 'train/*.parquet'))
VALID_PATHS = sorted(glob.glob(BASE_DIR + 'valid/*.parquet'))
TRAIN_PATHS, VALID_PATHS

We can see that our ratings are of only 0 and 1 after preprocessing.

In [ ]:
df = cudf.read_parquet(TRAIN_PATHS[0])
df.head()

In [ ]:
df.dtypes

In [ ]:
del df
gc.collect()

At this step we are going to generate config.pbtxt and we will save our workflow as a .pkl file to be able to load again to do transformation for the test (new coming) datasets at inference stage. This step actually does the serialization of the workflow that we created above using the training set.

In [ ]:
# converts a workflow to a triton mode 
MODEL_NAME_NVT = os.environ.get('MODEL_NAME_NVT', 'movielens_nvt_mh')
MODEL_PATH_TEMP_NVT = os.path.join(BASE_DIR, MODEL_NAME_NVT)

from nvtabular.inference.triton import generate_nvtabular_model
generate_nvtabular_model(workflow, MODEL_NAME_NVT, MODEL_PATH_TEMP_NVT)

Now we will train a TF MLP model using our preprocessed parquet files.

### Defining Hyperparameters

First, we define the data schema, and define our single-hot categorical features. Note, that we do not have any numerical input features. 

In [ ]:
BATCH_SIZE = 1024*32                            # Batch Size
CATEGORICAL_COLUMNS = ['movieId', 'userId']     # Single-hot
CATEGORICAL_MH_COLUMNS = ['genres']             # Multi-hot
NUMERIC_COLUMNS = []

TRAIN_PATHS = sorted(glob.glob(BASE_DIR + 'train/*.parquet')) # Output from ETL-with-NVTabular
VALID_PATHS = sorted(glob.glob(BASE_DIR + 'valid/*.parquet')) # Output from ETL-with-NVTabular

We can get the embedding input and output dimensions.

In [ ]:
EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(workflow)
EMBEDDING_TABLE_SHAPES

We will use emb_dim 64 for `movieId` and `userId` categorical features.

In [ ]:
EMBEDDING_TABLE_SHAPES = {'genres': (21, 16), 'movieId': (56586, 64), 'userId': (162542, 64)}

### Initializing NVTabular Data Loader for Tensorflow

The NVTabular data loader is initialized as usual and we specify both single-hot and multi-hot categorical features as cat_names. The data loader will automatically recognize the single/multi-hot columns and represent them accordingly.

In [ ]:
train_dataset_tf = KerasSequenceLoader(
    TRAIN_PATHS, # you could also use a glob pattern
    batch_size=BATCH_SIZE,
    label_names=['rating'],
    cat_names=CATEGORICAL_COLUMNS+CATEGORICAL_MH_COLUMNS,
    cont_names=NUMERIC_COLUMNS,
    engine='parquet',
    shuffle=True,
    buffer_size=0.06, # how many batches to load at once
    parts_per_chunk=1
)

valid_dataset_tf = KerasSequenceLoader(
    VALID_PATHS, # you could also use a glob pattern
    batch_size=BATCH_SIZE,
    label_names=['rating'],
    cat_names = CATEGORICAL_COLUMNS+CATEGORICAL_MH_COLUMNS,
    cont_names=NUMERIC_COLUMNS,
    engine='parquet',
    shuffle=False,
    buffer_size=0.06,
    parts_per_chunk=1
)

### Defining Neural Network Architecture

We will define a common neural network architecture for tabular data.

- Single-hot categorical features are fed into an Embedding Layer
- Each value of a multi-hot categorical features is fed into an Embedding Layer and the multiple Embedding outputs are combined via averaging
- The output of the Embedding Layers are concatenated
- The concatenated layers are fed through multiple feed-forward layers (Dense Layers with ReLU activations)
- The final output is a single number with sigmoid activation function

First, we will define some dictonary/lists for our network architecture.

In [ ]:
inputs = {}    # tf.keras.Input placeholders for each feature to be used
emb_layers = []# output of all embedding layers, which will be concatenated

We create `tf.keras.Input` tensors for all 4 input features.

In [ ]:
for col in CATEGORICAL_COLUMNS:
    inputs[col] =  tf.keras.Input(
        name=col,
        dtype=tf.int64,
        shape=(1,)
    )
# Note that we need two input tensors for multi-hot categorical features
for col in CATEGORICAL_MH_COLUMNS:
    inputs[col+'__values'] = tf.keras.Input(
        name=f"{col}__values", 
        dtype=tf.int64, 
        shape=(1,)
    )
    inputs[col+'__nnzs'] = tf.keras.Input(
        name=f"{col}__nnzs", 
        dtype=tf.int64, 
        shape=(1,)
    )

Next, we initialize Embedding Layers with `tf.feature_column.embedding_column`.

In [ ]:
for col in CATEGORICAL_COLUMNS+CATEGORICAL_MH_COLUMNS:
    emb_layers.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_identity(
                col, 
                EMBEDDING_TABLE_SHAPES[col][0]                    # Input dimension (vocab size)
            ), EMBEDDING_TABLE_SHAPES[col][1]                     # Embedding output dimension
        )
    )
emb_layers

NVTabular implemented a custom TensorFlow layer `layers.DenseFeatures`, which takes as an input the different `tf.Keras.Input` and pre-initialized `tf.feature_column` and automatically concatenates them into a flat tensor. `DenseFeatures` can handle numeric inputs, as well, but MovieLens does not provide numerical input features.

In [ ]:
emb_layer = layers.DenseFeatures(emb_layers)
x_emb_output = emb_layer(inputs)
x_emb_output

We can see that the output shape of the concatenated layer is equal to the sum of the individual Embedding output dimensions.


We add multiple Dense Layers. Finally, we initialize the `tf.keras.Model` and add the optimizer.

In [ ]:
x = tf.keras.layers.Dense(128, activation='relu')(x_emb_output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=x)
model.compile('sgd', 'binary_crossentropy')

### Training the deep learning model

We can train our model with `model.fit`.

In [ ]:
validation_callback = KerasSequenceValidater(valid_dataset_tf)

history = model.fit(train_dataset_tf, callbacks=[validation_callback], epochs=3)

We can save the trained TF model

In [ ]:
MODEL_NAME_TF = os.environ.get("MODEL_NAME_TF", "movielens_tf_mh")
MODEL_PATH_TEMP_TF = os.path.join(BASE_DIR, MODEL_NAME_TF, "1/model.savedmodel")

model.save(MODEL_PATH_TEMP_TF)

In [ ]:
# Creates an ensemble triton server model, where
#   model: The tensorflow model that should be served
#   workflow: The nvtabular workflow used in preprocessing
#   name: The base name of the various triton models

MODEL_NAME_ENSEMBLE = os.environ.get("MODEL_NAME_ENSEMBLE", "movielens_mh")
MODEL_PATH = os.environ.get("MODEL_PATH", "/model/models_multihot/")

from nvtabular.inference.triton import export_tensorflow_ensemble
export_tensorflow_ensemble(model, workflow, MODEL_NAME_ENSEMBLE, MODEL_PATH, ["rating"])

In [ ]:
#!apt-get install tree

Each model in a model repository must include a model configuration that provides required and optional information about the model. Next, Triton needs a config file to understand how to interpret the model.

In [ ]:
# !tree /model/models_multihot/

We will move to the next notebook, [movielens-multihot-inference](), to send request to the Triton Inference Server.